In [1]:
import pandas as pd
import numpy as np

In [2]:
base = pd.read_csv('Raw-Data.csv')

In [3]:
base.head()

,Country,Age,Gender,Symptoms,Experiencing_Symptoms,Severity,Contact
0,China,0-9,Male,"Fever,Tiredness,Dry-Cough,Difficulty-in-Breath...","Pains,Nasal-Congestion,Runny-Nose,Diarrhea",Mild,Yes
1,Italy,10-19,Female,"Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing","Pains,Nasal-Congestion,Runny-Nose",Moderate,No
2,Iran,20-24,Transgender,"Fever,Tiredness,Dry-Cough","Pains,Nasal-Congestion",Severe,Dont-Know
3,Republic of Korean,25-59,NaN,"Fever,Tiredness",Pains,None,NaN
4,France,60+,NaN,Fever,"Nasal-Congestion,Runny-Nose,Diarrhea",NaN,NaN


In [4]:
base_cleaned = pd.read_csv('Cleaned-Data.csv')

In [5]:
base_cleaned.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,China


In [6]:
print(base.shape, base_cleaned.shape)

(16, 7) (316800, 27)


In [7]:
pd.DataFrame(np.unique(base_cleaned.Country, return_counts=True))

,0,1,2,3,4,5,6,7,8,9
0,China,France,Germany,Iran,Italy,Other,Other-EUR,Republic of Korean,Spain,UAE
1,31680,31680,31680,31680,31680,31680,31680,31680,31680,31680


In [8]:
# UNir os dados de serveridade para poder usar como classificador.

In [9]:
join_columns = ['Severity_Severe', 'Severity_Moderate', 'Severity_Mild' ,'Severity_None']

# Join columns to use classifier
def join_dummies(df, columns, new_column_name='Join_columns'):
    join_columns = df[columns]
    new_df = []
    for i in range(len(join_columns)):
        ind = join_columns.iloc[i].idxmax()
        new_df.append(ind)

    df[new_column_name] = new_df

join_dummies(base_cleaned, join_columns, new_column_name='Severity')

In [10]:
# Testar se todos so possuem um mesmo (Medo dos dados terem problemas)
df = base_cleaned[join_columns]
for i in range(len(base_cleaned)):
    s = df.iloc[i].sum()
    if s != 1:
        print(df.iloc[i])

In [11]:
base_cleaned.Severity.value_counts()

Severity_Moderate    79200
Severity_Mild        79200
Severity_Severe      79200
Severity_None        79200
Name: Severity, dtype: int64

In [12]:
severity_base = base_cleaned.drop(join_columns, axis=1)
pd.set_option('display.max_columns', None)
severity_base.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Severity
0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,1,China,Severity_Mild
1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,China,Severity_Mild
2,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,China,Severity_Mild
3,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,1,China,Severity_Moderate
4,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,China,Severity_Moderate


In [13]:
# Separar dados de teste e treino

In [14]:
# Categorical Encode
import category_encoders as ce
encoder = ce.OrdinalEncoder(cols=['Severity', 'Country'], return_df=True)
encoder.fit(severity_base)

/home/beloin/anaconda3/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


OrdinalEncoder(cols=['Severity', 'Country'],
               mapping=[{'col': 'Severity', 'data_type': dtype('O'),
                         'mapping': Severity_Mild        1
Severity_Moderate    2
Severity_Severe      3
Severity_None        4
NaN                 -2
dtype: int64},
                        {'col': 'Country', 'data_type': dtype('O'),
                         'mapping': China                  1
Italy                  2
Iran                   3
Republic of Korean     4
France                 5
Spain                  6
Germany                7
UAE                    8
Other-EUR              9
Other                 10
NaN                   -2
dtype: int64}])

In [15]:
encoded_base = encoder.transform(severity_base)
encoded_base

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,None_Experiencing,Age_0-9,Age_10-19,Age_20-24,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Severity
0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,1
1,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,1,1
2,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,1,0,0,1,1
3,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,1,1,2
4,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,1,0,0,1,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,10,3
316796,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,10,3
316797,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,10,4
316798,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,10,4


In [35]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

no_country_base = severity_base.drop('Country',axis=1)
X = no_country_base.iloc[:,:22]
y = no_country_base.iloc[:,22]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

no_country_encoded_base = encoded_base.drop('Country',axis=1)
X_encoded = no_country_encoded_base.iloc[:,:22]
y_encoded = no_country_encoded_base.iloc[:,22]

X_encoded_train, X_encoded_test, y_encoded_train, y_encoded_test = train_test_split(X_encoded, y_encoded, test_size=.3)

tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
pred_tree = tree_model.predict(X_test)
acc_tree = accuracy_score(y_test, pred_tree)
cf_tree = confusion_matrix(y_test, pred_tree, )
acc_tree, cf_tree

(0.13114478114478115,
 array([[4883, 7927, 6218, 4858],
        [9292, 3422, 6176, 4820],
        [9357, 7099, 2442, 4810],
        [9413, 7025, 5581, 1717]]))

In [36]:
def test_model(model, x, y, x_test, y_test):
    model.fit(x, y)
    pred = model.predict(x_test)
    acc = accuracy_score(y_test, pred)
    cf = confusion_matrix(y_test, pred)
    return acc, cf

In [37]:
from sklearn.naive_bayes import GaussianNB
gaussian_model = GaussianNB()

test_model(gaussian_model, X_encoded_train, y_encoded_train, X_encoded_test, y_encoded_test)

(0.2473169191919192,
 array([[4659, 9203, 5270, 4502],
        [4686, 9289, 5314, 4479],
        [4709, 9289, 5174, 4530],
        [4823, 9440, 5290, 4383]]))

In [39]:
from sklearn.ensemble import RandomForestClassifier
random_model = RandomForestClassifier()
test_model(random_model, X_encoded_train, y_encoded_train, X_encoded_test, y_encoded_test)

(0.13170244107744108,
 array([[3118, 6707, 7058, 6751],
        [6879, 3042, 7001, 6846],
        [6864, 6781, 3233, 6824],
        [6962, 6754, 7096, 3124]]))

In [40]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier()
test_model(knn_model, X_encoded_train, y_encoded_train, X_encoded_test, y_encoded_test)

(0.21654040404040403,
 array([[7195, 7031, 5578, 3830],
        [8440, 5783, 5614, 3931],
        [8492, 6852, 4479, 3879],
        [8599, 6915, 5299, 3123]]))

In [27]:
len(X.columns)

22

In [43]:
y_encoded_train

27358     4
218779    3
117172    2
197525    2
170863    3
         ..
110102    1
20827     3
311451    2
231409    1
300769    1
Name: Severity, Length: 221760, dtype: int64

In [51]:
y_ann

221760

In [97]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

input_dim = len(X.columns)
input_dim

opt = Adam(learning_rate=0.03)

replace_dic = {1:0, 2:1, 3:2, 4:3}
y_ann = np.array(y_encoded_train)
for k, v in replace_dic.items():
    y_ann[y_ann == k] = v    


ann_model = Sequential()
ann_model.add(Dense(10, input_dim=input_dim, activation='relu'))
ann_model.add(Dense(12, activation='relu'))
ann_model.add(Dense(5, activation='softmax'))
ann_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
ann_model.fit(X_encoded_train, y_ann, epochs=120, batch_size=260)

Epoch 1/120
853/853 [==============================] - 2s 2ms/step - loss: 1.3994 - accuracy: 0.2499
Epoch 2/120
853/853 [==============================] - 1s 1ms/step - loss: 1.3921 - accuracy: 0.2491
Epoch 3/120
853/853 [==============================] - 1s 2ms/step - loss: 1.3919 - accuracy: 0.2512
Epoch 4/120
707/853 [=======================>......] - ETA: 0s - loss: 1.3925 - accuracy: 0.2504